#### IMPORT MODULES

In [51]:
import cohere
import pandas as pd
import requests
import datetime
import os,sys
sys.path.append('../scr')
import config
from tqdm import tqdm
from cProfile import label
from File_handler import FileHandler
file_data=FileHandler()
api_key = config.cohere_api["api_key"]

In [52]:
# Paste your API key here. Remember to not share publicly
#api from config

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [53]:
#data = file_data.dvc_get_data('../data/relations_dev.json', 'v3')
import json
#train data
with open("../data/relations_dev.json",'r') as read_file:
  train_df = pd.read_json(read_file.read())
#test data
test_df = pd.read_json('../data/relations_test.json')  

In [87]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   document   22 non-null     object
 1   tokens     22 non-null     object
 2   relations  22 non-null     object
 3   label      22 non-null     object
dtypes: object(4)
memory usage: 832.0+ bytes


In [54]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   document   11 non-null     object
 1   tokens     11 non-null     object
 2   relations  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


In [55]:
def pre_process_tokens(tokens):
    enti={}
    for t in tokens:
        
        if t['entityLabel'] in enti.keys():
            enti[t['entityLabel']] = enti[t['entityLabel']] +','+t['text']
        else:
            enti[t['entityLabel']] = t['text']
    label = ""
    for k, v in enti.items():
        label += k+':'+v+"\n"
    return label

In [56]:
#apply processing 
train_df['label'] = train_df['tokens'].apply(pre_process_tokens)
train_df

,document,tokens,relations,label
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG...",DIPLOMA:Bachelor\nDIPLOMA_MAJOR:Mechanical Eng...
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:10+ years,5+ years\nSKILLS:software..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP...",EXPERIENCE:3+ years\nSKILLS:Swift & Objective-C\n
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:8+ years,5+ years\nSKILLS:software ..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG...",DIPLOMA:BS\nDIPLOMA_MAJOR:Computer Science\nEX...
5,8+ years of development experience with Python...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:8+ years\nSKILLS:development,Python..."
6,Strong problem solving and analytical skills i...,"[{'text': '2+ years', 'start': 773, 'end': 781...","[{'child': 129, 'head': 125, 'relationLabel': ...",EXPERIENCE:2+ years\nSKILLS:performance modeli...
7,"Experience in backend services, systems, and d...","[{'text': 'BS/MS', 'start': 66, 'end': 71, 'to...","[{'child': 15, 'head': 11, 'relationLabel': 'D...",DIPLOMA:BS/MS\nDIPLOMA_MAJOR:Computer Science\...
8,experience: 5 + years of full-stack software d...,"[{'text': '5 + years', 'start': 12, 'end': 21,...","[{'child': 6, 'head': 2, 'relationLabel': 'EXP...","EXPERIENCE:5 + years,5+ years,5 + years,3+ yea..."
9,BS in Electrical Engineering or Computer Engin...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 2, 'head': 0, 'relationLabel': 'DEG...","DIPLOMA:BS,MS\nDIPLOMA_MAJOR:Electrical Engine..."


In [57]:
test_df['label'] = test_df['tokens'].apply(pre_process_tokens)
test_df

,document,tokens,relations,label
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:machine learning,..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...",EXPERIENCE:2+ years\nSKILLS:online advertising...
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","EXPERIENCE:5+ years,2+ years\nSKILLS:project m..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","DIPLOMA:MS,PhD\nDIPLOMA_MAJOR:Operations Resea..."
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","EXPERIENCE:12+ years\nSKILLS:research,Ph.D\nDI..."
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","EXPERIENCE:5+ years,3+ years,3+ years,5+ years..."


In [58]:
#extract entity from the train dataset
train_word = []
for i in range(train_df.shape[0]):
    entity = train_df.label.iloc[i]
    file_doc = train_df.document.iloc[i].replace("\n"," ")
    if len(entity)!=0:
        train_word.append(file_doc+"\n\nExtracted Text:" +
                        '\n'+entity+"----\n")
#store the entity on the file
with open('../data/extract_entity.txt','w') as f:
    for item in train_word:
        # write each item on a new line
        f.write("%s\n" % item.strip())
    print('Successful')

Successful


In [59]:
test_doc = test_df.document.apply(
    lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()

In [60]:
from pipeline import Pipeline
pipeline =Pipeline()

In [61]:
#The first thing is to extract all the data from our json
class cohereExtractor():
    def __init__(self, examples):
        self.examples = examples

    def make_prompt(self, example):
        prompt = self.examples + [example]
       
        return ("".join([str(exam) for exam in prompt]))

    def extract(self, example):
        extraction = co.generate(
            model='large',
            prompt=self.make_prompt(example),
            max_tokens=100,
            temperature=0.5,
            stop_sequences=["----"])
        
        return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor(train_doc[:6])

In [62]:

print(cohereMovieExtractor.make_prompt(test_doc[2]))

Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience

Extracted Text:
DIPLOMA:Bachelor
DIPLOMA_MAJOR:Mechanical Engineering,Physical Science
EXPERIENCE:3+ years
SKILLS:developing,fiber optic cables,connector related products
----
10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experie

In [63]:
results = []
for text in tqdm(test_doc):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

  0%|          | 0/11 [00:00<?, ?it/s]


DIPLOMA:Ph.D.
DIPLOMA_MAJOR:Computer Science
EXPERIENCE:1+ year(s),0+ year(s),0+ year(s)
SKILLS:machine learning,AI,computer science,statistics,applied mathematics,data science,research,publication
---


  9%|▉         | 1/11 [00:03<00:37,  3.78s/it]


EXPERIENCE:2+ years
SKILLS:online advertising,research,economic,statistics,political science,history,psychology
---


 18%|█▊        | 2/11 [00:07<00:32,  3.62s/it]

...

 82%|████████▏ | 9/11 [00:34<00:07,  3.73s/it]


DIPLOMA:PhD
DIPLOMA_MAJOR:Psychology
EXPERIENCE:3+ years
SKILLS:people research,organizations
---


 91%|█████████ | 10/11 [00:38<00:03,  3.70s/it]


EXPERIENCE:1+ year
SKILLS:research,AI,machine learning,computer science,statistics,applied mathematics,data science,research
---


100%|██████████| 11/11 [00:42<00:00,  3.85s/it]


In [64]:
pd.DataFrame(data={'text': test_doc, 'extracted_text': results})

,text,extracted_text
0,"Currently holding a faculty, industry, or gov...",\nDIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Scienc...
1,2+ years experience in the online advertising...,\nEXPERIENCE:2+ years\nSKILLS:online advertisi...
2,BA/BS 5+ years of program or project manageme...,"\nEXPERIENCE:5+ years,2+ years\nSKILLS:program..."
3,Currently enrolled in a full-time degree prog...,"\nEXPERIENCE:3+ years\nSKILLS:research,machine..."
4,Currently enrolled in a full-time degree prog...,\nDIPLOMA:Pursuing\nDIPLOMA_MAJOR:Computer Sci...
5,MS in a quantitative field such as Operations...,"\nEXPERIENCE:7+ years,5+ years\nSKILLS:MS in a..."
6,"Ph.D. with 5+ years of experience, MS with 7+...","\nEXPERIENCE:5+ years,7+ years,10+ years\nSKIL..."
7,Graduating with a Ph.D. in Computer Science o...,\nDIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Scienc...
8,"12+ years of research experience, in-house or...","\nEXPERIENCE:12+ years\nSKILLS:research,human-..."
9,"PhD with 3+ years, or MS with 5+ years of exp...",\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Psychology\nEXPER...


In [80]:
def clean_extracted(x):
    x = "".join(x)
    x = x.lower()
    x = x[:-3]
    x = x.lstrip()
    
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
    return x

In [81]:
def clean_test(x):
    x = "".join(x)
    x = x.lower()
    x = x.lstrip()
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
    return x

In [82]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['document'].apply(
            lambda x: "Job: "+x.replace("\n", " ")+'\n\nExtracted Text:').to_list()):
        extracted.append(str(i).strip())

In [76]:
# Save results
test_df['extracted_text'] = extracted
test_df['extracted_text'] = test_df['extracted_text'].apply(clean_extracted)
test_df['label'] = test_df['label'].apply(clean_test)
from difflib import SequenceMatcher
score = []
for i in range(test_df.shape[0]):
    
    score.append(SequenceMatcher(None,test_df['label'].iloc[i],
                          test_df['extracted_text'].iloc[i]).ratio())
# Compare the label to the extracted text
test_df['similarity_score'] = pd.DataFrame(score)

# Print the accuracy
print(f'Classification accuracy {test_df["similarity_score"].mean() *100}%')

Classification accuracy 43.026703753879936%


In [86]:
test_df[test_df['similarity_score']==0]

,document,tokens,relations,label,extracted_text,similarity_score,correct
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],,"diplomaph.d.,ms,bsdiploma_majorphysics,electri...",0.0,0


In [ ]:
=============///===================